In [48]:
import sys
sys.path.append('../')
import pickle
import numpy as np
np.random.seed(314159)

from metric_functions import compute_auc_from_scores, compute_accuracy
from sklearn.model_selection import train_test_split

In [49]:
train_size = 0.4

In [50]:
synth3_data = np.loadtxt(f'../synth_data/synth_0.3.csv', delimiter = ',')
synth4_data = np.loadtxt(f'../synth_data/synth_0.4.csv', delimiter = ',')

In [51]:
np.random.seed(314159)

X = synth3_data[:, :-1]
t = synth3_data[:, -1]

s3_X_train, s3_X_test, s3_t_train, s3_t_test = train_test_split(X, t, train_size = train_size, shuffle = True, stratify = t)

s3_t_train = s3_t_train.reshape(-1, 1)
s3_t_test = s3_t_test.reshape(-1, 1)

In [52]:
np.random.seed(314159)

X = synth4_data[:, :-1]
t = synth4_data[:, -1]

s4_X_train, s4_X_test, s4_t_train, s4_t_test = train_test_split(X, t, train_size = train_size, shuffle = True, stratify = t)

s4_t_train = s4_t_train.reshape(-1, 1)
s4_t_test = s4_t_test.reshape(-1, 1)

In [53]:
with open('../QA_bagged_results/synth-3/synth_0.3QA_models', 'rb') as f:
    s3_QA_models = pickle.load(f)

with open('../QA_bagged_results/synth-4/synth-4_bagged_models', 'rb') as f:
    s4_QA_models = pickle.load(f)

In [54]:
#For each model in each bag, calculate the AUROC and put it in a np array
s3_model_auc = []
s3_model_acc = []

for i, bag in enumerate(s3_QA_models):

    #Scores from each classifier in the bag
    bag_scores = []
    for j, model in enumerate(bag):

        #Each model's list of scores
        bag_scores.append(model.decision_function(s3_X_test))

    #Bag scores is a list of arrays length 72
    mean_scores = sum(bag_scores)/len(bag_scores[0])
    preds = np.sign(mean_scores)

    s3_model_auc.append(compute_auc_from_scores(mean_scores, s3_t_test))
    s3_model_acc.append(compute_accuracy(preds, s3_t_test))


In [55]:
#For each model in each bag, calculate the AUROC and put it in a np array
s4_model_auc = []
s4_model_acc = []

for i, bag in enumerate(s4_QA_models):

    #Scores from each classifier in the bag
    bag_scores = []
    for j, model in enumerate(bag):

        #Each model's list of scores
        bag_scores.append(model.decision_function(s4_X_test))

    #Bag scores is a list of arrays length 72
    mean_scores = sum(bag_scores)/len(bag_scores[0])
    preds = np.sign(mean_scores)

    s4_model_auc.append(compute_auc_from_scores(mean_scores, s4_t_test))
    s4_model_acc.append(compute_accuracy(preds, s4_t_test))


In [56]:
np.save('../QA_bagged_results/synth-3_QA_auroc', s3_model_auc)
np.save('../QA_bagged_results/synth-3_QA_accuracy', s3_model_acc)

np.save('../QA_bagged_results/synth-4_QA_auroc', s4_model_auc)
np.save('../QA_bagged_results/synth-4_QA_accuracy', s4_model_auc)